In [8]:
import pandas as pd
def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

link = "https://drive.google.com/file/d/1QMgcwm1rUhY14xb1fEfGyJYJ94QUeUYy/view?usp=drive_link"
direct_link = get_direct_link(link)
combined_df = pd.read_csv(direct_link)

players = pd.DataFrame(pd.concat([combined_df['Winner'],combined_df['Loser']]).unique())
players.columns = ['original_names']
players['search_query'] = players['original_names'] + " mens tennis wikipedia"

In [16]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

def get_first_search_result_title(query):
    query = urllib.parse.quote_plus(query)
    url = f"https://www.google.com/search?q={query}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        'Accept-Language': 'en-US,en;q=0.5'
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    first_result = soup.find("div", {"class": "yuRUbf"})
    if first_result:
        title = first_result.find("h3").text
        return title.strip()

    return None

count = 684
for index, row in players.iterrows():
    if index < count:
        continue
        
    query = row["search_query"]
    done_player_row = done_players[done_players['search_query'] == query]
    
    if not done_player_row.empty:
        players.at[index, "wiki_title"] = done_player_row.iloc[0]['wiki_title']
    else:
        result_title = get_first_search_result_title(query)
        if not result_title:
            print(count, index, row['original_names'])
            break
        players.at[index, "wiki_title"] = result_title if result_title else "No search results found"
    
    print(index)


684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700


In [18]:
players["wiki_title"] = players["wiki_title"].str.replace(' - Wikipedia', '')

In [19]:
players.to_csv("mens_tennis_players.csv", encoding='utf-8', index=False)